In [1]:
import os

import evaluate
import numpy as np
from datasets import load_dataset
from dotenv import load_dotenv
from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments, Trainer, \
    DataCollatorWithPadding, AutoModelForSequenceClassification, AutoTokenizer

In [2]:
load_dotenv()

True

## Load the model. I used `roberta-large`.

In [3]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large', token=os.getenv('HUGGINGFACE_TOKEN'))
model = AutoModelForSequenceClassification.from_pretrained('roberta-large',
                                                           trust_remote_code=False,
                                                           token=os.getenv('HUGGINGFACE_TOKEN'))

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

C:\Users\aksid\PycharmProjects\ReviewCategoriser\venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksid\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Use PEFT for effeciency.

In [4]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    bias='none',
    lora_dropout=0.1,
)

In [5]:
model = get_peft_model(model, lora_config)

In [6]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): M

### EDA

In [7]:
imdb = load_dataset("imdb")
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
def preprocess_function(examples):
    # Tokenize the reviews
    text_tokenized = tokenizer(examples['text'], padding='max_length', truncation=True)
    return text_tokenized


tokenized_train = imdb['train'].map(preprocess_function, batched=True)
tokenized_test = imdb['test'].map(preprocess_function, batched=True)
tokenized_unsupervised = imdb['unsupervised'].map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [10]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [11]:
def compute_metrics(eval_pred):
    load_accuracy = evaluate.load("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy, }

In [12]:
training_args = TrainingArguments(
    output_dir="./saved_model/training",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    warmup_steps=1000,
    do_train=True,
    do_eval=True,
    bf16=True,
)

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\aksid\PycharmProjects\ReviewCategoriser\venv\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
%%time

training_result = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.164000,0.143813,0.959600
2,0.164200,0.134780,0.963520
3,0.139900,0.140287,0.963840


CPU times: total: 18min 36s
Wall time: 54min 52s


In [16]:
# compute train results
metrics = training_result.metrics
max_train_samples = len(tokenized_train)
metrics["train_samples"] = min(max_train_samples, len(tokenized_train))

In [17]:
# compute evaluation results
metrics = trainer.evaluate()
max_val_samples = len(tokenized_test)
metrics["eval_samples"] = min(max_val_samples, len(tokenized_test))

In [18]:
metrics

{'eval_loss': 0.13478049635887146,
 'eval_accuracy': 0.96352,
 'eval_runtime': 294.8682,
 'eval_samples_per_second': 84.784,
 'eval_steps_per_second': 10.598,
 'epoch': 3.0,
 'eval_samples': 25000}

### Performance Metrics

| eval_loss | eval_accuracy | eval_runtime | eval_samples_per_second | eval_steps_per_second | epoc | eval_samples |
|---|---|---|---|---|---|---|
| 0.13478049635887146 | 0.96352 | 294.8682 | 84.784 | 10.598 | 3.0 | 25000 |

### Test!

In [19]:
import random

N = len(tokenized_unsupervised)

idx = random.randint(1, N)

example = tokenized_unsupervised[idx]
text = example['text']
text

'My town is a small one and we can only have the chance to see the big Hollywood productions.I had to wait for months to see this movie and I found it at Blockbusters yesterday.The cast is a dream cast for me I always liked Peter Mullan after My name is Joe.Milla Jovovich is an actress which I really find passionate.When the film was over I could not believe how bad it was.Meaningless details,bad picture quality and unbelievably bad story. I am so sorry for the cast and the director.'

In [20]:
tokenised = tokenizer(text, return_tensors='pt')
model = model.to('cpu')
res = model(**tokenised)

In [21]:
import torch.nn.functional as F

probabilities = F.softmax(res.logits, dim=1)
predicted_class = probabilities.argmax(dim=1)
predicted_class[0]

tensor(0)

In [22]:
trainer.save_model('saved_model/roberta-large-peft')

In [23]:
model.num_parameters()

357199876